In [11]:
import flash
import torch
from flash.tabular import TabularClassificationData, TabularClassifier
import pickle
import os
import pandas as pd
import numpy as np
from flash.core.data.utilities.classification import SingleNumericTargetFormatter, SingleBinaryTargetFormatter

In [2]:
with open(os.path.join("..", "data", "npys_data", "X_keys.pkl"), "rb") as fp:
    X_keys = pickle.load(fp)

In [19]:
# 1. Create the DataModule
datamodule = TabularClassificationData.from_csv(
    numerical_fields=X_keys,
    target_fields="target",
    train_file=os.path.join("..", "data", "processed_files", "flash", "X_TRAIN_FR_RUS.csv"),
    val_file=os.path.join("..", "data", "processed_files", "flash", "X_VAL_FR_RUS.csv"),
    test_file=os.path.join("..", "data", "processed_files", "flash", "X_TEST_FR_RUS.csv"),
    batch_size=64,
    num_workers=20,
    target_formatter=SingleNumericTargetFormatter(num_classes=4)
)

/home/crop_dev/miniconda/envs/flash/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3526: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
# 2. Build the task
model = TabularClassifier.from_data(datamodule, backbone="tabtransformer")

Using 'tabtransformer' provided by manujosephv/PyTorch Tabular (https://github.com/manujosephv/pytorch_tabular).


In [21]:
datamodule.num_classes

4

In [22]:
# 3. Create the trainer and train the model
trainer = flash.Trainer(max_epochs=3, accelerator='gpu', gpus=[2])
trainer.fit(model, datamodule=datamodule)

/home/crop_dev/miniconda/envs/flash/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=[2])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[2])` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name          | Type                  | Params
--------------------------------------------------------
0 | train_metrics | ModuleDict            | 0     
1 | val_metrics   | ModuleDict            | 0     
2 | test_metrics  | ModuleDict            | 0     
3 | adapter       | PytorchTabularAdapter | 272 K 
--------------------------------------------------------
272 K     Trainable params
0         Non-trainable params
272 K     Total params
1.089     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/crop_dev/miniconda/envs/flash/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (15) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [8]:
predictions = trainer.test(model, datamodule=datamodule)

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      valid_accuracy       │    0.8029999732971191     │
│        valid_loss         │            nan            │
└───────────────────────────┴───────────────────────────┘

[{'valid_loss': nan, 'valid_accuracy': 0.8029999732971191}]
